In [279]:
import numpy as np
import itertools
import time  
import json
import requests

# Setup.

tot = {
    'A': 9, 'D': 5, 'E': 8, 'I': 5, 'N': 6, 'R': 8, 'S': 8, 'T': 9,
    'G': 3, 'L': 5, 'O': 6, 
    'B': 2, 'F': 2, 'H': 2, 'K': 3, 'M': 3, 'V': 2, 
    'P': 2, 'U': 3, 'Å': 2, 'Ä': 2, 'Ö': 2,
    'J': 1, 'Y': 1,
    'C': 1, 'X': 1, 'Z': 1, '?': 2
}

p = {
    'A': 1, 'D': 1, 'E': 1, 'I': 1, 'N': 1, 'R': 1, 'S': 1, 'T': 1,
    'G': 2, 'L': 2, 'O': 2, 
    'B': 3, 'F': 3, 'H': 3, 'K': 3, 'M': 3, 'V': 3, 
    'P': 4, 'U': 4, 'Å': 4, 'Ä': 4, 'Ö': 4,
    'J': 7, 'Y': 7,
    'C': 8, 'X': 8, 'Z': 8, '?': 0
}

do = []
to = []
db = []
tb = []
zero_point_tiles = []

word_list = open("saol_out_3.txt", encoding="UTF-8").readlines()
tmp = []
for w in word_list:
    tmp.append(w.replace('\n', ''))

word_set = set(tmp)

In [ ]:
# Fetches board, finds and plays best move. Need to add sessionid.

headers = {
            "User-Agent": "WebFeudClient/3.0.17 (Android 10)",
            "Host": "api.wordfeud.com",
            "Connection": "Keep-Alive",
            "Accept-Encoding": "gzip",
            "Cookie": f"sessionid={}",
        }

response = requests.get(
    f"https://api.wordfeud.com/wf/user/games/detail/?known_tile_points=&known_boards=",
    headers=headers,
    verify=True)

parsed = response.json()

game_id = parsed['content']['games'][0]['id']
response = requests.get(
    f"https://api.wordfeud.com/wf/games/{game_id}/?known_tile_points=&known_boards=",
    headers=headers,
    verify=True)

parsed = response.json()    
h = parsed['content']['games'][0]['players'][1]['rack']

grid = grid = parsed['content']['boards'][0]['board']
tiles = parsed['content']['games'][0]['tiles']
print("tiles", tiles)
for x in range(15):
    for y in range(15):
        if grid[x][y] == 1:
            db.append([x,y])
        if grid[x][y] == 2:
            tb.append([x,y])
        if grid[x][y] == 3:
            do.append([x,y])
        if grid[x][y] == 4:
            to.append([x,y])
        grid[x][y] = str(0)
for tile in tiles:
    grid[tile[1]][tile[0]] = tile[2]
    if tile[3]:
        print("zerotile", [tile[1], tile[0]])
        zero_point_tiles.append([tile[1],tile[0]])
print(np.matrix(grid))

# TODO: Add support for zerotile in hand.

all_over = []
for i in range(1, len(h) + 1):
    c = itertools.combinations(h, i)

    for combo in c:
        all_over.append(list(combo))

perm_list = []   
for ra in all_over:
    it = itertools.permutations(ra)
    for sad in it:
        perm_list.append(list(sad))

perm_list = list(perm_list for perm_list, _ in itertools.groupby(perm_list))

start_time = time.time()
valid_moves = solve()
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.6f} seconds")

headers = {
    "User-Agent": "WebFeudClient/3.0.17 (Android 10)",
    "Content-Type": "application/json; charset=UTF-8",
    "Content-Length": "0",
    "Host": "api.wordfeud.com",
    "Connection": "Keep-Alive",
    "Accept-Encoding": "gzip",
    "Cookie": f"sessionid={}",
}

data = max(valid_moves)[1]
response = requests.post(
    f"https://api.wordfeud.com/wf/game/{game_id}/move/",
    data=data.encode("UTF-8"),
    headers=headers,
    verify=True,
)

parsed = response.json()
print(parsed)

In [182]:
def calc_points_main(clear_pos, current_letters, y, x):
    points = 0
    
    for pos, b in zip(clear_pos, current_letters):
        mult = 1
        if pos in db:
            mult = 2
        elif pos in tb:
            mult = 3
        points += p[b] * mult
    
    word = get_adjacent_letters(y, x)
    pre_letters = list(set(word) - set(current_letters))
    
    for b in pre_letters:
        points += p[b]
    
    for pos, b in zip(clear_pos, current_letters):
        if pos in do:
            points *= 2
        elif pos in to:
            points *= 3
            
    return points

def calc_points_and_validate_not_main(is_transposed, clear_pos, current_letters): #TODO: Remove global grids?
    global grid

    points = 0
    for pos, b in zip(clear_pos, current_letters):
        if is_transposed:
            y, x = pos[1], pos[0]
            word = get_adjacent_letters(x, y)
        else:
            y, x = pos[0], pos[1]
            
            grid = [[grid[j][i] for j in range(len(grid))] for i in range(len(grid[0]))]
            word = get_adjacent_letters(x, y)
            grid = [[grid[j][i] for j in range(len(grid))] for i in range(len(grid[0]))]

        if word not in word_list and len(word) != 1:
            clear(clear_pos)

            return False, 0
        
        if len(word) != 1:
            for b2 in word:
                points += p[b2] 
                
            if pos in db:
                points += p[b2] 
            elif pos in tb:
                points += p[b2] * 2 
            elif pos in do:
                points *= 2
            elif pos in to:
                points *= 3

    return True, points

def place_letters_row(clear_pos, y, x, current_letters):
    global grid
    
    x_i = 0
    for b in current_letters:

        if x + x_i > 15 - 1:
            clear(clear_pos)
            
            return False

        while x + x_i < 15 - 1 and grid[y][x + x_i] != '0':
            x_i += 1

        grid[y][x + x_i] = b
        clear_pos.append([y, x + x_i])
        x_i += 1

    return True

def place_letters_column(clear_pos, y, x, current_letters):
    global grid
    
    y_i = 0
    for b in current_letters:

        if y + y_i > 15 - 1:
            clear(clear_pos)
            
            return False

        while y + y_i < 15 - 1 and grid[y + y_i][x] != '0':
            y_i += 1

        grid[y + y_i][x] = b
        clear_pos.append([y + y_i, x])
        y_i += 1

    return True

def get_valid_start_row(current_letters):
    global grid
    
    valid_start_row = []
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if grid[y][x] != '0':
                if x + 1 > 15 - 1:
                    break
                
                if grid[y][x + 1] == '0':
                    valid_start_row.append([y, x + 1])

                for i in range(len(current_letters)):
                    x_tmp = x - len(current_letters) + i
                    if x_tmp >= 0 and grid[y][x_tmp] == '0':
                        valid_start_row.append([y, x_tmp])
      
    return valid_start_row
 
def get_valid_start_column(current_letters):
    global grid
    
    valid_start_column = []
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if grid[y][x] != '0':
                if y + 1 > 15 - 1:
                    break
                if grid[y + 1][x] == '0':
                    valid_start_column.append([y + 1, x])

                for i in range(len(current_letters)):
                    y_tmp = y - len(current_letters) + i
                    if y_tmp >= 0 and grid[y_tmp][x] == '0':
                        valid_start_column.append([y_tmp, x])
      
    return valid_start_column    

def get_existing_pos():
    global grid
    
    existing_pos = []
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if grid[y][x] != '0':
                existing_pos.append([y, x])
                
    return existing_pos
        
def clear(clear_pos):
    global grid
    
    for pos in clear_pos:
        grid[pos[0]][pos[1]] = '0'

def get_adjacent_letters(y, x):
    global grid
    
    row = grid[y]
    
    left_index = x
    right_index = x

    # Move left until a space or the beginning of the string is found INCLUDES Start
    while left_index > 0 and row[left_index - 1] != '0':
        left_index -= 1

    # Move right until a space or the end of the string is found
    while right_index < len(row) - 1 and row[right_index + 1] != '0':
        right_index += 1

    return ''.join(row[left_index:right_index + 1])

In [ ]:
def solve():
    global grid
    global perm_list
    
    existing_pos = get_existing_pos()

    valid_moves = []
    for current_letters in perm_list:
        #print("current_letters:", current_letters)
        
        valid_start_row = get_valid_start_row(current_letters)
        if not existing_pos:
            for i in range(len(current_letters)):
                valid_start_row.append([7, 7 - len(current_letters) + i + 1])
            
        for cords in valid_start_row:
            y, x = cords[0], cords[1]
            clear_pos = []
            
            if not place_letters_row(clear_pos, y, x, current_letters):
                clear(clear_pos)
                continue
            
            l_r_word = get_adjacent_letters(y, x)

            if l_r_word not in word_set:
                clear(clear_pos)
                continue
            
            pos = clear_pos[0]
            points = calc_points_main(clear_pos, current_letters, pos[0], pos[1])
            if existing_pos:
                ok, points_not_main = calc_points_and_validate_not_main(False, clear_pos, current_letters)
                
                if not ok:
                    clear(clear_pos)
                    continue
                
                points += points_not_main

            letters_put_at_position = []
            for cl, cp in zip(current_letters, clear_pos):
                letters_put_at_position.append([cp[1], cp[0], cl.upper(), False])
            valid_moves.append([points, json.dumps({
                "words": [l_r_word.upper()],
                "ruleset": 4,
                "move": letters_put_at_position
            }, ensure_ascii=False)])
            #print(np.matrix(grid), "\n", l_r_word)
            clear(clear_pos)
             
        valid_start_column = get_valid_start_column(current_letters)
        if not existing_pos:
            for i in range(len(current_letters)):
                valid_start_column.append([7 - len(current_letters) + i + 1, 7])
                
        for cords in valid_start_column:
            y, x = cords[0], cords[1]
            clear_pos = []
            
            if not place_letters_column(clear_pos, y, x, current_letters):
                clear(clear_pos)
                continue

            grid = [[grid[j][i] for j in range(len(grid))] for i in range(len(grid[0]))]
            u_d_word = get_adjacent_letters(x, y)
            grid = [[grid[j][i] for j in range(len(grid))] for i in range(len(grid[0]))]
            
            if u_d_word not in word_set:
                clear(clear_pos)
                continue
                
            grid = [[grid[j][i] for j in range(len(grid))] for i in range(len(grid[0]))]
            pos = clear_pos[0]
            points = calc_points_main(clear_pos, current_letters, pos[1], pos[0])
            grid = [[grid[j][i] for j in range(len(grid))] for i in range(len(grid[0]))]
            
            if existing_pos:
                ok, points_not_main = calc_points_and_validate_not_main(True, clear_pos, current_letters)
                
                if not ok:
                    clear(clear_pos)
                    continue
                
                points += points_not_main
                
            letters_put_at_position = []
            for cl, cp in zip(current_letters, clear_pos):
                letters_put_at_position.append([cp[1], cp[0], cl.upper(), False])
            valid_moves.append([points, json.dumps({
                "words": [u_d_word.upper()],
                "ruleset": 4,
                "move": letters_put_at_position
            }, ensure_ascii=False)])
            #print(np.matrix(grid), "\n", u_d_word)
            clear(clear_pos)
            
    return valid_moves